# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

G.-D. Marleau  ->  G.-D. Marleau  |  ['G.-D. Marleau']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
S. Li  ->  S. Li  |  ['S. Li']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Li  ->  J. Li  |  ['J. Li']


M. Benisty  ->  M. Benisty  |  ['M. Benisty']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
T. Henning  ->  T. Henning  |  ['T. Henning']
S. Scheithauer  ->  S. Scheithauer  |  ['S. Scheithauer']
Arxiv has 86 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2408.01491


extracting tarball to tmp_2408.01491...

 done.


G.-D. Marleau  ->  G.-D. Marleau  |  ['G.-D. Marleau']


...

extracting tarball to tmp_2408.02305...

 done.
Retrieving document from  https://arxiv.org/e-print/2408.02374


extracting tarball to tmp_2408.02374...

 done.


Found 58 bibliographic references in tmp_2408.02374/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.01491-b31b1b.svg)](https://arxiv.org/abs/2408.01491) | **Accreting companion occurrence rates using a new method to compute emission-line survey sensitivity: Application to the H-alpha Giant Accreting Protoplanet Survey (GAPlanetS)**  |
|| C. Plunkett, K. B. Follette, <mark>G.-D. Marleau</mark>, E. Nielsen |
|*Appeared on*| *2024-08-06*|
|*Comments*| *Submitted to ApJ. Comments welcome. 18 pages, 7 figures*|
|**Abstract**|            A key scientific goal of exoplanet surveys is to characterize the underlying population of planets in the local galaxy. In particular, the properties of accreting protoplanets can inform the rates and physical processes of planet formation. We develop a novel method to compute sensitivity to protoplanets in emission-line direct-imaging surveys, enabling estimates of protoplanet population properties under various planetary accretion and formation theories. In this work, we specialize to the case of H-alpha and investigate three formation models governing the planetary-mass-to-mass-accretion-rate power law, and two accretion models that describe the scaling between total accretion luminosity and observable H-alpha line luminosity. We apply our method to the results of the Magellan Giant Accreting Protoplanet Survey (GAPlanetS) to place the first constraints on accreting companion occurrence rates in systems with transitional circumstellar disks. We compute the posterior probability for transitional disk systems to host an accreting companion (-8< log MMdot MJ^2/yr < -2) within 2 arcseconds (~200 au). Across accretion models, we find consistent accreting companion rates, with median and one-sigma credible intervals of 0.15 (+0.18, -0.10) and 0.19 (+0.23, -0.12). Our technique enables studying protoplanet populations under flexible assumptions about planet formation. This formalism provides the statistical underpinning necessary for protoplanet surveys to discriminate among formation and accretion theories for planets and brown dwarfs.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.02374-b31b1b.svg)](https://arxiv.org/abs/2408.02374) | **The GRAVITY young stellar object survey XIV : Investigating the magnetospheric accretion-ejection processes in S CrA N**  |
|| G. Collaboration, et al. -- incl., <mark>M. Benisty</mark>, <mark>P. Garcia</mark>, <mark>T. Henning</mark>, <mark>S. Scheithauer</mark> |
|*Appeared on*| *2024-08-06*|
|*Comments*| **|
|**Abstract**|            The dust- and gas-rich protoplanetary disks around young stellar systems play a key role in star and planet formation. While considerable progress has recently been made in probing these disks on large scales of a few tens of astronomical units (au), the central au needs to be more investigated. We aim at unveiling the physical processes at play in the innermost regions of the strongly accreting T Tauri Star S CrA N by means of near-infrared interferometric observations. The K-band continuum emission is well reproduced with an azimuthally-modulated dusty ring. As the star alone cannot explain the size of this sublimation front, we propose that magnetospheric accretion is an important dust-heating mechanism leading to this continuum emission. The differential analysis of the Hydrogen Br$\gamma$ line is in agreement with radiative transfer models combining magnetospheric accretion and disk winds. Our observations support an origin of the Br$\gamma$ line from a combination of (variable) accretion-ejection processes in the inner disk region.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.01665-b31b1b.svg)](https://arxiv.org/abs/2408.01665) | **Cosmological perturbations in the energy-momentum squared gravity theory: constraints from gravitational wave standard sirens and redshift space distortions**  |
|| Q.-M. Fu, <mark>X. Zhang</mark> |
|*Appeared on*| *2024-08-06*|
|*Comments*| *11 pages, 2 figures*|
|**Abstract**|            We investigate the linear cosmological perturbations in the context of the so-called energy-momentum squared gravity (EMSG) theory. Recent researches show that the EMSG theory can reproduce viable background cosmological evolution comparable to $\Lambda$CDM, while the matter-dominated era exhibits slight distinctions. In this paper, we mainly focus on the power-law EMSG models and derive the equations for the linear cosmological perturbations. We explore the propagation of the gravitational wave (GW) and the growth of matter density perturbation at the first order, and estimate the model parameters from the simulated GW data and the observed redshift space distortion data. Our analysis reveals that the model parameters should be small and positive in $1\sigma$ confidence interval, which indicates that the theory is in good agreement with the observational data and can be regarded as an alternative for the standard cosmological model.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.02171-b31b1b.svg)](https://arxiv.org/abs/2408.02171) | **J-PLUS: Beyond Spectroscopy III. Stellar Parameters and Elemental-abundance Ratios for Five Million Stars from DR3**  |
|| Y. Huang, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-08-06*|
|*Comments*| *31 pages, 20 figures, 3 tables, accepeted by ApJ*|
|**Abstract**|            We present a catalog of stellar parameters (effective temperature $T_{\rm eff}$, surface gravity $\log g$, age, and metallicity [Fe/H]) and elemental-abundance ratios ([C/Fe], [Mg/Fe], and [$\alpha$/Fe]) for some five million stars (4.5 million dwarfs and 0.5 million giants stars) in the Milky Way, based on stellar colors from the Javalambre Photometric Local Universe Survey (J-PLUS) DR3 and \textit{Gaia} EDR3. These estimates are obtained through the construction of a large spectroscopic training set with parameters and abundances adjusted to uniform scales, and trained with a Kernel Principal Component Analysis. Owing to the seven narrow/medium-band filters employed by J-PLUS, we obtain precisions in the abundance estimates that are as good or better than derived from medium-resolution spectroscopy for stars covering a wide range of the parameter space: 0.10-0.20 dex for [Fe/H] and [C/Fe], and 0.05 dex for [Mg/Fe] and [$\alpha$/Fe]. Moreover, systematic errors due to the influence of molecular carbon bands on previous photometric-metallicity estimates (which only included two narrow/medium-band blue filters) have now been removed, resulting in photometric-metallicity estimates down to [Fe/H] $\sim -4.0$, with typical uncertainties of 0.25 dex and 0.40 dex for dwarfs and giants, respectively. This large photometric sample should prove useful for the exploration of the assembly and chemical-evolution history of our Galaxy.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.02305-b31b1b.svg)](https://arxiv.org/abs/2408.02305) | **Star Formation, Nebulae, and Active Galactic Nuclei in CLASH Brightest Cluster Galaxies. I. Dependence on Core Entropy of Intracluster Medium**  |
|| A. Levitskiy, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-08-06*|
|*Comments*| *39 Pages, 18 Figures*|
|**Abstract**|            We set the stage for reassessing how star formation, emission-line nebulae, and active galactic nuclei (AGNs) in brightest cluster galaxies (BCGs) depend on the thermodynamics of the intracluster medium (ICM). Our work is based on the 25 clusters observed in the CLASH program for which the aforementioned attributes in their BCGs can be well scrutinized, as has the thermodynamics of their ICM. Nine of these BCGs display complex UV morphologies tracing recent star formation, whereas the remaining 16 are characterized by a relatively compact central UV enhancement. Here, we show definitively that three of the latter BCGs also display star formation, whereas the diffuse UV of the remaining 13 is entirely consistent with old low-mass stars. The overall results support the previously established dependence of star formation and nebulae in BCGs on an "excess core entropy," K$_{0}$, for the ICM: all 11 clusters with K$_{0}$ $\leq$ 24 keV cm$^{2}$ (but only one of 14 clusters with K$_{0}$ $\geq$ 42 keV cm$^{2}$) host star-forming BCGs that almost if not always possess nebulae. Instead of an entropy floor, we show that K$_{0}$ reflects the degree to which the radial entropy profile decreases inward within $\sim$100 kpc rather than (except perhaps at large K$_{0}$) actually flattening: clusters with lower ICM entropies and hence shorter cooling times at their cores preferentially host BCGs displaying star formation, nebulae, and more radio-luminous AGNs. Nearly all BCGs possess detectable AGNs, however, indicating multiple pathways for fuelling their AGNs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.01937-b31b1b.svg)](https://arxiv.org/abs/2408.01937) | **Inflight Performance and Calibrations of the Lyman-alpha Solar Telescope on board the Advanced Space-based Solar Observatory**  |
|| B. Chen, et al. -- incl., <mark>J. Li</mark>, <mark>X. Zhang</mark>, <mark>S. Li</mark> |
|*Appeared on*| *2024-08-06*|
|*Comments*| *Solar Physics (ASO-S mission topical collection), accepted*|
|**Abstract**|            The Lyman-alpha Solar Telescope (LST) on board the Advanced Space-based Solar Observatory (ASO-S) is the first payload to image the full solar disk and the solar corona in both white-light (WL) and ultraviolet (UV) H I Lya, extending up to 2.5 solar radii (Rs). Since the launch of the ASO-S on 9 October 2022, LST has captured various significant solar activities including flares, prominences, coronal mass ejections (CMEs). LST covers different passbands of 121.6 nm, 360 nm and 700 nm. The Lya Solar Disk Imager (SDI) has a field of view (FOV) of 38.4 arcmin and a spatial resolution of around 9.5 arcsec, while the White-Light Solar Telescope (WST) has a FOV of 38.43 arcmin and a spatial resolution of around 3.0 arcsec. The FOV of the Lya Solar Corona Imager (SCI) reaches 81.1 arcmin and its spatial resolution is 4.3 arcsec. The stray-light level in the 700 nm waveband is about 7.8e-6 MSB (mean solar brightness) at 1.1 Rs and 7.6e-7 MSB at 2.5 Rs, and in the Lya waveband it is around 4.3e-3 MSB at 1.1 Rs and 4.1e-4 MSB at 2.5 Rs. This article will detail the results from on-orbit tests and calibrations.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2408.01491/./figs/flowchart.png', 'tmp_2408.01491/./figs/completeness_rast.png', 'tmp_2408.01491/./figs/posteriors_conf.png']
copying  tmp_2408.01491/./figs/flowchart.png to _build/html/
copying  tmp_2408.01491/./figs/completeness_rast.png to _build/html/
copying  tmp_2408.01491/./figs/posteriors_conf.png to _build/html/
exported in  _build/html/2408.01491.md
    + _build/html/tmp_2408.01491/./figs/flowchart.png
    + _build/html/tmp_2408.01491/./figs/completeness_rast.png
    + _build/html/tmp_2408.01491/./figs/posteriors_conf.png
found figures ['tmp_2408.02374/./Data.png', 'tmp_2408.02374/./BRG_Shifts.png', 'tmp_2408.02374/./schema_scales.png']
copying  tmp_2408.02374/./Data.png to _build/html/
copying  tmp_2408.02374/./BRG_Shifts.png to _build/html/
copying  tmp_2408.02374/./schema_scales.png to _build/html/
exported in  _build/html/2408.02374.md
    + _build/html/tmp_2408.02374/./Data.png
    + _build/html/tmp_2408.02374/./BRG_Shifts.png
    + _build/html/tmp_24

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\Ha}{H\alpha\xspace}$
$\newcommand{\Lacc}{\ensuremath{L_{\mathrm{acc}}}\xspace}$
$\newcommand{\LHa}{\ensuremath{L_{\mathrm{H}\alpha}}\xspace}$
$\newcommand{\MMd}{\ensuremath{M\dot M}\xspace}$
$\newcommand{\MdtoM}{\ensuremath{\dot M\textrm{--}M}\xspace}$
$\newcommand{\LHatoLacc}{\ensuremath{L_{\mathrm{H}\alpha}\textrm{--}L_{\mathrm{acc}}}\xspace}$
$\newcommand{\cp}[1]{\textcolor{orange}{CP:  \textit{#1}}}$
$\newcommand{\kf}[1]{\textcolor{blue}{KF:  \textit{#1}}}$
$\newcommand{\MMdunits}{\ensuremath{M_J^2 \rm{yr}^{-1}}\xspace}$</div>



<div id="title">

# Accreting companion occurrence rates using a new method to compute emission-line survey sensitivity: application to the H$\alpha$ Giant Accreting Protoplanet Survey (GAPlanetS)

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.01491-b31b1b.svg)](https://arxiv.org/abs/2408.01491)<mark>Appeared on: 2024-08-06</mark> -  _Submitted to ApJ. Comments welcome. 18 pages, 7 figures_

</div>
<div id="authors">

C. Plunkett, K. B. Follette, <mark>G.-D. Marleau</mark>, E. Nielsen

</div>
<div id="abstract">

**Abstract:** A key scientific goal of exoplanet surveys is to characterize the underlying population of planets in the local galaxy. In particular, the properties of accreting _proto_ planets can inform the rates and physical processes of planet formation. We develop a novel method to compute sensitivity to protoplanets in emission-line direct-imaging surveys, enabling estimates of protoplanet population properties under various planetary accretion and formation theories. In this work, we specialize to the case of $\Ha$ and investigate three formation models governing the planetary-mass-to-mass-accretion-rate power law, and two accretion models that describe the scaling between total accretion luminosity and observable $\Ha$ line luminosity. We apply our method to the results of the Magellan Giant Accreting Protoplanet Survey (GAPlanetS) to place the first constraints on accreting companion occurrence rates in systems with transitional circumstellar disks. We compute the posterior probability for transitional disk systems to host an accreting companion ( $-8\leq\log \MMd[\MMdunits]\leq-2$ ) within 2 arcseconds ( $\sim200$ au). Across accretion models, we find consistent accreting companion rates, with median and one-sigma credible intervals of $0.15^{+0.18}_{-0.10}$ and $0.19^{+0.23}_{-0.12}$ . Our technique enables studying protoplanet populations under flexible assumptions about planet formation. This formalism provides the statistical underpinning necessary for protoplanet surveys to discriminate among formation and accretion theories for planets and brown dwarfs.

</div>

<div id="div_fig1">

<img src="tmp_2408.01491/./figs/flowchart.png" alt="Fig3" width="100%"/>

**Figure 3. -** Steps to estimate direct-imaging survey completeness to accreting companions. At each physical parameter set $(a,M)$, we simulate $10^4$ accreting companions. First, we sample mass accretion rates $\dot M$ using an assumed model for the $\MdtoM$ scaling. Step A shows the models in Section \ref{ssec:formpar}; the stellar model is in purple and the planetary in green. Next, we convert each $\MMd$ to $\Lacc$(Section \ref{ssec:gravinfall}). We then compute the observable $\LHa$ based on a model for the accretion scaling; Step B shows the two models in Section \ref{ssec:accpar}. We obtain the projected separation distribution by sampling companion orbital parameters as in Section \ref{ssec:orbits}; Step C shows one such distribution for an object at $a=100$ au. Finally, we compare the simulated companions to the survey star's contrast curve to estimate the detectable fraction. Step D shows two examples of 100 brown dwarfs $(a,M) = (100 \mathrm{au},  26 M_J)$ around PDS70; the left panel shows companions simulated using both "stellar" models, while the right uses both "planetary" models. We repeat across a grid in $(a.M)$ for each combination of models to obtain the entire completeness maps (Step E). (*fig:flowchart*)

</div>
<div id="div_fig2">

<img src="tmp_2408.01491/./figs/completeness_rast.png" alt="Fig4" width="100%"/>

**Figure 4. -** Completeness to accreting companions for the 14 stars surveyed by GAPlanetS as a function of semimajor axis [au] and mass $[M_J]$ for the four combinations of models. The color scale shows the depth of search, or the number of stars to which the survey is complete. The row dictates the $\MdtoM$ relation, with the top the stellar \citepalias{Betti2023} model and bottom the planetary \citepalias{Stamatellos2015} model. The column defines the $\LHa$toLacc scaling, with the left the stellar \citepalias{Alcala2017} and right the planetary \citepalias{Aoyama2021} scaling. Contours show completeness to 2.5, 5, 7.5, and 10 (out of 14) stars in the sample. (*fig:completeness*)

</div>
<div id="div_fig3">

<img src="tmp_2408.01491/./figs/posteriors_conf.png" alt="Fig6" width="100%"/>

**Figure 6. -** Posteriors on accreting object occurrence rate around stars hosting transitional disks, $f_p$, for different subsets of the GAPlanetS detections. In all, we compare the stellar (purple) and planetary (green) accretion scaling relations. The legends give the median and one-sigma uncertainties (16$^{\textrm{th}}$ and 84$^{\textrm{th}}$ percentiles), also marked by dashed lines. _Left:_ posteriors on $f_p$ using the three confident detections. _Center:_ posteriors on the rate of accreting companions _inside_ disk gaps, excluding HD100453 B, which lies outside the gap. _Right:_ posteriors on the accreting _protoplanet_ rate, excluding the low-mass stars HD100453 B and HD142527 B. (*fig:posteriors_conf*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.01491"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\muas}{\upmu\rm{as}}$
$\newcommand{\ms}{m s^{-1}}$
$\newcommand{\kms}{km.s^{-1}}$
$\newcommand{\vsini}{v\sin i}$
$\newcommand{\vsinia}{v\sin i_{\rm A}}$
$\newcommand{\vsinib}{v\sin i_{\rm B}}$
$\newcommand{\vrad}{v_{\rm rad}}$
$\newcommand{\vmic}{v_{\rm mic}}$
$\newcommand{\vmac}{v_{\rm th}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\msunpyr}{M_{\odot}.yr^{-1}}$
$\newcommand{\Lsun}{L_\odot}$
$\newcommand{\rsun}{R_{\odot}}$
$\newcommand{\bl}{B_{\ell}}$
$\newcommand{\bd}{B_{\rm D}}$
$\newcommand{\ddeg}{^{\circ}}$
$\newcommand{\te}{T_{\rm eff}}$
$\newcommand{\logg}{\log{g}}$
$\newcommand{\prot}{P_{\rm rot}}$
$\newcommand{\Brgam}{Br\gamma}$
$\newcommand{\Th}{^\text{th}}$</div>



<div id="title">

# The GRAVITY young stellar object survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.02374-b31b1b.svg)](https://arxiv.org/abs/2408.02374)<mark>Appeared on: 2024-08-06</mark> - 

</div>
<div id="authors">

G. Collaboration, et al. -- incl., <mark>M. Benisty</mark>, <mark>P. Garcia</mark>, <mark>T. Henning</mark>, <mark>S. Scheithauer</mark>

</div>
<div id="abstract">

**Abstract:** The dust- and gas-rich protoplanetary disks around young stellar systems play a key role in star and planet formation. While considerable progress has recently been made in probing these disks on large scales of a few tens of astronomical units (au), the central au needs to be more investigated. We aim at unveiling the physical processes at play in the innermost regions of the strongly accreting T Tauri Star S CrA N by means of near-infrared interferometric observations. As recent spectropolarimetric observations suggested that S CrA N might undergo intense ejection processes, we focus on the accretion-ejection phenomena and on the star-disk interaction region. We obtained interferometric observations with VLTI/GRAVITY in the K-band during two consecutive nights in August 2022. The analysis of the continuum emission, coupled with the differential analysis acrossthe $\Brgam$ line, allows us to constrain the morphology of the dust and gas distribution in the innermost regions of S CrA N and to investigate their temporal variability. These observations are compared to magnetospheric accretion/ejection models of T Tauri stars and to previous observations to describe the physical processes operating in these regions. The K-band continuum emission is well reproduced with an azimuthally-modulated dusty ring with a half-light radius of 0.24 au ( $\sim$ 20 $R_*$ ), an inclination of $\sim$ 30 $\ddeg$ , and a position angle of $\sim$ 150 $\ddeg$ . As the star alone cannot explain such a large sublimation front, we propose that magnetospheric accretion is an important dust-heating mechanism leading to this continuum emission. The $\Brgam$ emitting region (0.05-0.06 au; 5-7 $R_*$ ) is found to be more compact than the continuum, and of the order or larger than the size of the magnetospheric truncation radius. The on-sky displacements across the $\Brgam$ spectral channels are aligned along a position angle offset by 45 $\ddeg$ from the disk, and extend up to 2 $R_*$ . This is in agreement with radiative transfer models combining magnetospheric accretion and disk winds. These on-sky displacements remain unchanged from one night to the other, while the line flux decreases by 13 \% , suggesting a dominant contribution of wind for the origin of the $\Brgam$ line. Our observations support an origin of the $\Brgam$ line from a combination of (variable) accretion-ejection processes in the inner disk region.

</div>

<div id="div_fig1">

<img src="tmp_2408.02374/./Data.png" alt="Fig7" width="100%"/>

**Figure 7. -** $\Brgam$ differential interferometric observables of S CrA N: Squared differential visibilities (left); Differential phases (middle); u-v plane coverage (upper-right); Line-to-continuum flux ratio in the reference frame of the star, LSR- tellurics-corrected (lower-right). Each color used in the u-v coverage plot corresponds to an interferometric baseline and is consistent from one panel to the other. For the interferometric observables, the plots include the quantities for each frame (solid lines), the median for August 19 (circles), and for August 20 (squares). Uncertainties are given in all plots (vertical bars), and are smaller than the symbol if not visible. Grey shaded regions correspond to the velocities that are not considered for our $\Brgam$ line analysis (see text for details). (*fig:BrGam_Data*)

</div>
<div id="div_fig2">

<img src="tmp_2408.02374/./BRG_Shifts.png" alt="Fig8" width="100%"/>

**Figure 8. -** Asymmetry of the $\Brgam$ emitting region of S CrA N. Left : pure-line differential phases. The pure line differential phases are depicted by blue-to-red color coded symbols for August 19 (circles), and for August 20 (squares), while the observed phases for each individual observation (similar to Fig. \ref{fig:BrGam_Data}) are depicted in solid lines, with their color corresponding to their baseline. Right : on-sky photocenter shifts with respect to the continuum photocenter (in (0, 0) as a function of velocity (coded from blue to red colors).
    The light yellow cones correspond to the uncertainties on the major axis orientation of the inner dusty disk. The background symbols are the displacements obtained from individual files.  (*fig:shifts*)

</div>
<div id="div_fig3">

<img src="tmp_2408.02374/./schema_scales.png" alt="Fig9" width="100%"/>

**Figure 9. -** S CrA N at different scales. Right: Representation of the star-disk interaction region (not to scale), with the characteristic sizes derived in this work. {The obliquity of the magnetosphere is just illustrative as no consolidated value exists}. Top-left: SPHERE polarized intensity image from [Zhang, Ginski and Huang (2023)](), and the directions toward the HH objects associated to S CrA. We also reported the PA of the dusty disk measured from the K-band continuum and the orientation of photocenter shifts as yellow, red and blue cones, respectively. Dashed cones highlight the suggested outflow emission (see text). Bottom-left: ALMA $^{12}$CO(2-1) first moment map from [Gupta, Miotello and Manara (2023)](). The cone is reported from the SPHERE observations, and the HH82 direction is indicated. Both images have been rotated to have the same orientation as the sketch, where the North and East directions are shown. (*fig:schema_scales*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.02374"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

152  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
